# Navie-Bayes on ionosphere dataset

**OBS**: [dataset information](https://archive.ics.uci.edu/dataset/52/ionosphere)

***

**Additional Information**
This radar data was collected by a system in Goose Bay, Labrador.  This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts.  See the paper for more details.  The targets were free electrons in the ionosphere. "Good" radar returns are those showing evidence of some type of structure in the ionosphere.  "Bad" returns are those that do not; their signals pass through the ionosphere.  

Received signals were processed using an autocorrelation function whose arguments are the time of a pulse and the pulse number.  There were 17 pulse numbers for the Goose Bay system.  Instances in this databse are described by 2 attributes per pulse number, corresponding to the complex values returned by the function resulting from the complex electromagnetic signal.

**Has Missing Values?**
No

***

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
df = pd.read_csv(url, header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [5]:
# OBS: in the original method it was used 'as_matrix' for extracting the data, but is deprecated
X = df.drop([34], axis = 1).values
y = df[34].values

print(X.shape)
print(y.shape)

(351, 34)
(351,)


In [8]:
# Train on data. No parameters to adjust
from sklearn.naive_bayes import GaussianNB       ### Because continuous data
from sklearn.model_selection import (
    StratifiedKFold, 
    cross_val_score, 
    train_test_split
)
import sklearn.neighbors as nb

# OBS: deleted 'random_state' parameter since its use is no longer needed
cv = StratifiedKFold(
    n_splits = 10, 
) 

gnb = GaussianNB()
cv_scores = cross_val_score(
    gnb,
    X = X,
    y = y,
    cv = cv
)
np.mean(cv_scores)  # Mean score of the executions

0.8803174603174604

In [25]:
from sklearn.model_selection import cross_val_predict  
from sklearn.metrics import accuracy_score, confusion_matrix

predicted = cross_val_predict(
    estimator = GaussianNB(), 
    X = X,
    y = y,  
    cv = cv
)  

print(confusion_matrix(y, predicted))  # Confusion matrix
print(accuracy_score(y, predicted))    # Accuracy compute using both, true value and prediction

[[ 92  34]
 [  8 217]]
0.8803418803418803


In [10]:
from sklearn.metrics import classification_report

print(classification_report(y, predicted))

              precision    recall  f1-score   support

           b       0.92      0.73      0.81       126
           g       0.86      0.96      0.91       225

    accuracy                           0.88       351
   macro avg       0.89      0.85      0.86       351
weighted avg       0.88      0.88      0.88       351



***
## Adjusting probability threshold

In [18]:
# Focus on predict class 1
y2 = np.zeros((y.shape))  # Create vector of zeros (empty) and pupulate with 1s and 0s to represents
                          # the positions where the elements of each class are, we will be focusing on class "b" (1)
y2[y == "b"] = 1
y2[y == "g"] = 0

# 'Unpacking' the new variables
(X_train, X_test,  y_train, y_test) = train_test_split(X, y2, test_size = .3, random_state = 1)

In [26]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def filterp(th,ProbClass1):
    """Given a threshold "th" and a set of probabilities of belonging to class 1 "ProbClass1", 
    return predictions.""" 
    y = np.zeros(ProbClass1.shape[0])
    for i,v in enumerate(ProbClass1):
        if ProbClass1[i]>th:
            y[i]=1
    return y  

clf = GaussianNB()
lth=[]

# We do a 10-fold cross-validation with 10 iterations
kf = StratifiedKFold(
    n_splits = 20, 
    shuffle = True, 
    random_state = 42
)

# Now we compute the threshold by iterating the data we have
for train_index, test_index in kf.split(X_train, y_train):
    
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train2, y_test2 = y2[train_index], y2[test_index]

    # Train with the training data of the iteration 
    clf.fit(X_train2, y_train2)
    # Obtaining probability predictions for test data of the iteration
    probs = clf.predict_proba(X_test2)
    # Collect probabilities of belonging to class 1
    ProbClass1 = probs[:,1]
    # Sort probabilities and generate pairs (threshold, f1-for-that-threshold) 
    res = np.array([[th,f1_score(y_test2,filterp(th,ProbClass1),pos_label=1)] for th in np.sort(ProbClass1)])
    
    """
    # Uncomment the following lines if you want to plot at each iteration how f1-score 
    # evolves increasing the threshold 
    plt.plot(res[:,0],res[:,1])
    plt.show()
    """
    
    # Find the threshold that has maximum value of f1-score
    maxF = np.max(res[:,1])
    pl = np.argmax(res[:,1])
    optimal_th = res[pl,0]
    
    # Store the optimal threshold found for the current iteration
    lth.append(optimal_th)

# Compute the average threshold for all 10 iterations    
threshold = np.mean(lth)
print(f"Selected threshold in 10-fold cross validation: {np.round(threshold, 6)}")

Selected threshold in 10-fold cross validation: 0.244704


In [27]:
# Train a classifier with the whole training data 
clf = GaussianNB()
clf.fit(X_train, y_train)

# Obtain probabilities for data on test set
probs = clf.predict_proba(X_test)

# Generate predictions using probabilities and threshold found on 10 folds cross-validation
prediction = filterp(threshold,probs[:,1])

print(classification_report(y_test, prediction))
# Ignore warnings explaining that in some iterations f1 score is 0

              precision    recall  f1-score   support

         0.0       0.83      0.96      0.89        70
         1.0       0.88      0.61      0.72        36

    accuracy                           0.84       106
   macro avg       0.85      0.78      0.80       106
weighted avg       0.85      0.84      0.83       106



In [17]:
clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         0.0       0.82      0.96      0.88        70
         1.0       0.88      0.58      0.70        36

    accuracy                           0.83       106
   macro avg       0.85      0.77      0.79       106
weighted avg       0.84      0.83      0.82       106

